In [1]:
from datetime import datetime
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pylab import rcParams

# set maximum display rows as 200
pd.set_option('display.max_rows', 200)

plt.style.use('ggplot') # use ggplot styling
# displays graphs in the notebook
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

In [3]:
df =pd.read_sql_table('microwave', 'sqlite:///dissertation.db') 

In [4]:
df_copy = df.copy()# group by half an hour, and change kWh to Wh in order to enlarge the number
df_half = pd.DataFrame ({'Half_Wh':(df_copy.groupby(pd.Grouper(freq='30T',key='time'))['kWh'].sum().ffill()*1000)})

In [7]:
# subset 80% of dataset for train and test, 20% for validation  
subset_num = 4368
df_train_test = df_half[['Half_Wh']][:subset_num]
df_valid = df_half[['Half_Wh']][subset_num:]

# transform 
train_test=df_train_test.values
train_test=train_test.astype('float32')

valid=df_valid.values
valid=valid.astype('float32')

In [8]:
# split into train and test sets
train_size = int(len(train_test) * 0.65)
test_size = len(train_test) - train_size
train, test = train_test[0:train_size], train_test[train_size:len(train_test)]
print(len(train), len(test))

3276 1092


In [9]:
# manipulate data to x,and y
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [10]:
# create train and test dataset, and vaild dataset
look_back=96
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
validX, validY = create_dataset(valid, look_back)

In [21]:
# create and fit Multilayer Perceptron model
model = Sequential()
model.add(Dense(193, input_dim=look_back, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mse','mae','mape'])
history = model.fit(trainX, trainY, epochs=200, batch_size=10, verbose=2)

Epoch 1/200
 - 1s - loss: 842.5965 - mse: 842.5967 - mae: 12.8811 - mape: 5819348992.0000
Epoch 2/200
 - 0s - loss: 694.5995 - mse: 694.5996 - mae: 10.5991 - mape: 4283358720.0000
Epoch 3/200
 - 0s - loss: 615.1984 - mse: 615.1984 - mae: 9.9810 - mape: 3966208000.0000
Epoch 4/200
 - 0s - loss: 575.7996 - mse: 575.7998 - mae: 9.4894 - mape: 3700289536.0000
Epoch 5/200
 - 0s - loss: 477.9145 - mse: 477.9145 - mae: 8.5054 - mape: 3165790720.0000
Epoch 6/200
 - 0s - loss: 419.5395 - mse: 419.5396 - mae: 7.9995 - mape: 3128278016.0000
Epoch 7/200
 - 0s - loss: 317.0561 - mse: 317.0560 - mae: 7.1117 - mape: 2838246656.0000
Epoch 8/200
 - 0s - loss: 262.6185 - mse: 262.6184 - mae: 6.3914 - mape: 2498208512.0000
Epoch 9/200
 - 0s - loss: 222.8485 - mse: 222.8485 - mae: 5.8379 - mape: 2413360384.0000
Epoch 10/200
 - 0s - loss: 212.4714 - mse: 212.4714 - mae: 5.6533 - mape: 2206342656.0000
Epoch 11/200
 - 0s - loss: 190.3493 - mse: 190.3495 - mae: 5.3652 - mape: 2101767808.0000
Epoch 12/200
 - 0

Epoch 95/200
 - 0s - loss: 15.7825 - mse: 15.7825 - mae: 1.2742 - mape: 297990784.0000
Epoch 96/200
 - 0s - loss: 17.9248 - mse: 17.9248 - mae: 1.2900 - mape: 322421440.0000
Epoch 97/200
 - 0s - loss: 15.4615 - mse: 15.4615 - mae: 1.2137 - mape: 286712320.0000
Epoch 98/200
 - 0s - loss: 18.9578 - mse: 18.9578 - mae: 1.3471 - mape: 309154848.0000
Epoch 99/200
 - 1s - loss: 22.0015 - mse: 22.0015 - mae: 1.3885 - mape: 309518720.0000
Epoch 100/200
 - 1s - loss: 23.2900 - mse: 23.2900 - mae: 1.3458 - mape: 299759872.0000
Epoch 101/200
 - 0s - loss: 17.3135 - mse: 17.3135 - mae: 1.2105 - mape: 275553120.0000
Epoch 102/200
 - 0s - loss: 17.5499 - mse: 17.5499 - mae: 1.2393 - mape: 281234336.0000
Epoch 103/200
 - 0s - loss: 17.3348 - mse: 17.3348 - mae: 1.1515 - mape: 237466592.0000
Epoch 104/200
 - 0s - loss: 21.4249 - mse: 21.4249 - mae: 1.2351 - mape: 244869104.0000
Epoch 105/200
 - 0s - loss: 16.7253 - mse: 16.7253 - mae: 1.1816 - mape: 255248624.0000
Epoch 106/200
 - 0s - loss: 11.4060 -

Epoch 189/200
 - 0s - loss: 9.3261 - mse: 9.3261 - mae: 0.7435 - mape: 78008136.0000
Epoch 190/200
 - 0s - loss: 9.9512 - mse: 9.9512 - mae: 0.7824 - mape: 66587072.0000
Epoch 191/200
 - 0s - loss: 8.9523 - mse: 8.9523 - mae: 0.7552 - mape: 66438008.0000
Epoch 192/200
 - 0s - loss: 6.5534 - mse: 6.5534 - mae: 0.6296 - mape: 51423940.0000
Epoch 193/200
 - 0s - loss: 8.6491 - mse: 8.6491 - mae: 0.7691 - mape: 73760312.0000
Epoch 194/200
 - 0s - loss: 10.0245 - mse: 10.0245 - mae: 0.7969 - mape: 80205848.0000
Epoch 195/200
 - 0s - loss: 13.8686 - mse: 13.8686 - mae: 0.9147 - mape: 76979920.0000
Epoch 196/200
 - 0s - loss: 16.0810 - mse: 16.0810 - mae: 0.9650 - mape: 94330104.0000
Epoch 197/200
 - 0s - loss: 13.3481 - mse: 13.3481 - mae: 0.9367 - mape: 88009848.0000
Epoch 198/200
 - 1s - loss: 10.0008 - mse: 10.0008 - mae: 0.7840 - mape: 67242808.0000
Epoch 199/200
 - 0s - loss: 8.1804 - mse: 8.1804 - mae: 0.7067 - mape: 63509388.0000
Epoch 200/200
 - 0s - loss: 8.6303 - mse: 8.6303 - mae:

In [17]:
# model evalute
trainScore = model.evaluate(trainX, trainY)
testScore = model.evaluate(testX, testY)
validScore = model.evaluate(validX, validY)

print('Train MSE: ', trainScore[1], 'MAE: ', trainScore[2])
print('Test MSE: ', testScore[1], 'MAE: ', testScore[2])
print('Valid MSE: ', validScore[1], 'MAE: ', validScore[2])

3599/3599 [==============================] - 0s 35us/step
Train MSE:  10.263174057006836 MAE:  0.8711609244346619
Test MSE:  958.6543579101562 MAE:  8.958436012268066
Valid MSE:  173.7445068359375 MAE:  1.9164400100708008


In [18]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
validPredict = model.predict(validX)
nextPredict = model.predict(testX[-1:,:])

In [19]:
print(nextPredict,df_valid[0:1].values)

[[0.4260016]] [[0.]]


In [20]:
from sklearn.metrics import r2_score
r2=r2_score(testY,testPredict)

#mape = np.mean(np.abs(testPredict - testY)/np.abs(testY))  # MAPE

print('R2: ', r2)

R2:  0.005574169060084233


In [ ]:
# with lockdown
mad_mean_ratio =8.958*1092/df_half['Half_Wh'][4368-1092:4368].sum() 
mad_mean_ratio